In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\neteera\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\neteera\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import sklearn
import math
from datetime import datetime
import pandas as pd

In [8]:
# Haversine function to calculate the distance between two points on the Earth's surface
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Output is the shortest distance between 2 points (in kilometers)

# User class to store carpooling participant information
class User:
    def __init__(self, user_id, name, car_model, fuel_efficiency, start_longitude, start_latitude, end_longitude, end_latitude, start_point_name, end_point_name, distance, travel_time, start_time, end_time, travel_days):
        self.user_id = user_id
        self.name = name
        self.car_model = car_model  # New attribute for car model
        self.fuel_efficiency = fuel_efficiency  # New attribute for fuel efficiency (km/liter)
        self.start_longitude = start_longitude
        self.start_latitude = start_latitude
        self.end_longitude = end_longitude
        self.end_latitude = end_latitude
        self.start_point_name = start_point_name
        self.end_point_name = end_point_name
        self.distance = distance  # distance willing to travel from start to end
        self.travel_time = travel_time
        self.start_time = datetime.strptime(start_time, '%H:%M')  # Start time as HH:MM
        self.end_time = datetime.strptime(end_time, '%H:%M')  # End time as HH:MM
        self.travel_days = travel_days  # New attribute for the days of travel (list of days, e.g., ["Monday", "Wednesday"])

    def __repr__(self):
        return f"User({self.name})"
    
def calculate_fuel_cost(distance, fuel_efficiency, fuel_price_per_liter=40):
    """
    Calculate the fuel cost based on distance, fuel efficiency, and fuel price per liter.
    """
    liters_needed = distance / fuel_efficiency
    return liters_needed * fuel_price_per_liter

# Function to find overlapping travel days
def overlapping_days(days1, days2):
    return list(set(days1) & set(days2))

# Matching function to find potential carpool matches
def find_matches(users, distance_threshold=1.0, time_threshold=30):
    matches = []
    
    for i, user1 in enumerate(users):
        for user2 in users[i+1:]:
            common_days = overlapping_days(user1.travel_days, user2.travel_days)
            if not common_days:
                continue  # Skip if there are no common travel days
            
            start_distance = haversine(user1.start_longitude, user1.start_latitude, user2.start_longitude, user2.start_latitude)
            end_distance = haversine(user1.end_longitude, user1.end_latitude, user2.end_longitude, user2.end_latitude)
            
            if start_distance <= distance_threshold and end_distance <= distance_threshold:
                # Check if travel time overlap is within threshold
                start_time_diff = abs((user1.start_time - user2.start_time).total_seconds() / 60)
                end_time_diff = abs((user1.end_time - user2.end_time).total_seconds() / 60)
                
                if start_time_diff <= time_threshold and end_time_diff <= time_threshold:
                    match_time = (max(user1.start_time, user2.start_time), min(user1.end_time, user2.end_time))
                    matches.append({
                        'user_id_person1': user1.user_id,
                        'person1_name': user1.name,
                        'user_id_person2': user2.user_id,
                        'person2_name': user2.name,
                        'start_latitude': user1.start_latitude,
                        'start_longitude': user1.start_longitude,
                        'start_point_name': user1.start_point_name,
                        'end_latitude': user1.end_latitude,
                        'end_longitude': user1.end_longitude,
                        'end_point_name': user1.end_point_name,
                        'start_time': user1.start_time.strftime('%H:%M'),
                        'end_time': user1.end_time.strftime('%H:%M'),
                        'match_time': f"{match_time[0].strftime('%H:%M')} - {match_time[1].strftime('%H:%M')}",
                        'common_days': common_days  # Added common travel days to match result
                    })
    
    return pd.DataFrame(matches)

# Function to create a cost table based on matches and fuel efficiency
def create_cost_table(users, matches_df, fuel_price_per_liter=40):
    cost_data = []
    
    for _, row in matches_df.iterrows():
        user1 = next(user for user in users if user.user_id == row['user_id_person1'])
        user2 = next(user for user in users if user.user_id == row['user_id_person2'])
        
        distance = user1.distance
        cost1 = calculate_fuel_cost(distance, user1.fuel_efficiency, fuel_price_per_liter)
        cost2 = calculate_fuel_cost(distance, user2.fuel_efficiency, fuel_price_per_liter)
        
        cost_data.append({
            'user_id_person1': user1.user_id,
            'person1_name': user1.name,
            'car_model_person1': user1.car_model,
            'fuel_efficiency_person1': user1.fuel_efficiency,
            'fuel_cost_person1': cost1,
            'user_id_person2': user2.user_id,
            'person2_name': user2.name,
            'car_model_person2': user2.car_model,
            'fuel_efficiency_person2': user2.fuel_efficiency,
            'fuel_cost_person2': cost2,
            'total_cost': cost1 + cost2,
            'common_days': row['common_days']  # Added common travel days to cost result
        })
    
    return pd.DataFrame(cost_data)

# Example usage
users = [
    User(1, "Alice", "Sedan", 15, 100.0, 13.7, 100.2, 13.8, "Start A", "End A", 5, 30, "08:00", "09:00", ["Monday", "Wednesday"]),
    User(2, "Bob", "SUV", 10, 100.1, 13.75, 100.25, 13.85, "Start B", "End B", 6, 35, "08:15", "09:15", ["Monday", "Friday"]),
    User(3, "Charlie", "Hatchback", 18, 101.0, 14.0, 101.1, 14.1, "Start C", "End C", 8, 45, "07:45", "08:45", ["Tuesday", "Wednesday"]),
    User(4, "Sally", "BMW", 18, 101.0, 14.0, 101.1, 14.1, "Start C", "End C", 8, 45, "07:45", "08:45", ["Thursday", "Sunday"]),
    User(5, "Jane", "Toyata", 18, 101.0, 14.0, 101.1, 14.1, "Start C", "End C", 8, 45, "07:45", "08:45", ["Tuesday", "Thursday"]),
]

matches_df = find_matches(users, distance_threshold=13.0, time_threshold=60)
cost_df = create_cost_table(users, matches_df, fuel_price_per_liter=40)

# Display the resulting DataFrame
print(cost_df)
print(matches_df)


   user_id_person1 person1_name car_model_person1  fuel_efficiency_person1  \
0                1        Alice             Sedan                       15   
1                3      Charlie         Hatchback                       18   
2                4        Sally               BMW                       18   

   fuel_cost_person1  user_id_person2 person2_name car_model_person2  \
0              13.33                2          Bob               SUV   
1              17.78                5         Jane            Toyata   
2              17.78                5         Jane            Toyata   

   fuel_efficiency_person2  fuel_cost_person2  total_cost common_days  
0                       10              20.00       33.33    [Monday]  
1                       18              17.78       35.56   [Tuesday]  
2                       18              17.78       35.56  [Thursday]  
   user_id_person1 person1_name  user_id_person2 person2_name  start_latitude  \
0                1        Ali

In [9]:
from IPython.display import display
# Use display to show the DataFrames as tables
display(cost_df)
display(matches_df)

,user_id_person1,person1_name,car_model_person1,fuel_efficiency_person1,fuel_cost_person1,user_id_person2,person2_name,car_model_person2,fuel_efficiency_person2,fuel_cost_person2,total_cost,common_days
0,1,Alice,Sedan,15,13.33,2,Bob,SUV,10,20.00,33.33,[Monday]
1,3,Charlie,Hatchback,18,17.78,5,Jane,Toyata,18,17.78,35.56,[Tuesday]
2,4,Sally,BMW,18,17.78,5,Jane,Toyata,18,17.78,35.56,[Thursday]


,user_id_person1,person1_name,user_id_person2,person2_name,start_latitude,start_longitude,start_point_name,end_latitude,end_longitude,end_point_name,start_time,end_time,match_time,common_days
0,1,Alice,2,Bob,13.70,100.00,Start A,13.80,100.20,End A,08:00,09:00,08:15 - 09:00,[Monday]
1,3,Charlie,5,Jane,14.00,101.00,Start C,14.10,101.10,End C,07:45,08:45,07:45 - 08:45,[Tuesday]
2,4,Sally,5,Jane,14.00,101.00,Start C,14.10,101.10,End C,07:45,08:45,07:45 - 08:45,[Thursday]


In [11]:
import pandas as pd
from IPython.display import display

# Assuming cost_df and matches_df are your DataFrames
# Merging the two DataFrames on 'user_id_person1' and 'user_id_person2'

combined_df = pd.merge(matches_df, cost_df, on=['user_id_person1', 'user_id_person2'])

# Optionally, set display options for better formatting in the notebook
pd.set_option('display.float_format', '{:.2f}'.format)

# Display the combined DataFrame as a table
display(combined_df)


,user_id_person1,person1_name_x,user_id_person2,person2_name_x,start_latitude,start_longitude,start_point_name,end_latitude,end_longitude,end_point_name,...,person1_name_y,car_model_person1,fuel_efficiency_person1,fuel_cost_person1,person2_name_y,car_model_person2,fuel_efficiency_person2,fuel_cost_person2,total_cost,common_days_y
0,1,Alice,2,Bob,13.70,100.00,Start A,13.80,100.20,End A,...,Alice,Sedan,15,13.33,Bob,SUV,10,20.00,33.33,[Monday]
1,3,Charlie,5,Jane,14.00,101.00,Start C,14.10,101.10,End C,...,Charlie,Hatchback,18,17.78,Jane,Toyata,18,17.78,35.56,[Tuesday]
2,4,Sally,5,Jane,14.00,101.00,Start C,14.10,101.10,End C,...,Sally,BMW,18,17.78,Jane,Toyata,18,17.78,35.56,[Thursday]
